In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

import os
import re
import io
import itertools
import pprint

from bokeh.io import output_notebook, show
from bokeh.layouts import gridplot
from bokeh.plotting import figure
from bokeh.models import Range1d, Legend
from bokeh.palettes import all_palettes

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr
import pymongo

import climatools.lblnew.bestfit_params as bestfits
from climatools.lblnew import setup_bestfit, setup_overlap
import climatools.lblnew.pipeline as pipe_lblnew
import climatools.cliradlw.setup as setup_cliradlw
import climatools.cliradlw.pipeline as pipe_cliradlw
from climatools.cliradlw.utils import mapband_new2old

import climatools.html.html as climahtml
from climatools.lblnew.dataio import *
from climatools.plot.plot import *

import IPython.display as display

In [2]:
output_notebook()

Loading BokehJS ...

In [3]:
def make_query(param=None):
    return {'param.' + name: value for name, value in param.items()}   

In [4]:
def show_html(s): display.display(display.HTML(s))
    
def show_markdown(s): display.display(display.Markdown(s))

In [5]:
def load_output_file(path_csv):
    '''
    Load lblnew output .csv file to xarray.Dataset
    
    Parameters
    ----------
    path_csv: str
              Path to the .csv file to be loaded.
    ds: xarray.Dataset
        Data in the input file in the form of an xarray.Dataset.
    '''
    toindex = ['i', 'band', 'pressure', 'igg', 'g']    
    df = pd.read_csv(path_csv, sep=r'\s+')
    df = df.set_index([i for i in toindex if i in df.columns])
    df = df.rename(columns={'sfu': 'flug',
                            'sfd': 'fldg',
                            'fnet': 'fnetg',
                            'coolr': 'coolrg'})
    ds = xr.Dataset.from_dataframe(df)

    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    return ds

In [6]:
def fmt_cool(ds_in):
    ds = ds_in.copy(deep=True)
    dims = ['igg', 'g', 'i']
    for dim in dims:
        if dim in ds:
            if ds[dim].shape == (1,): ds = ds.squeeze(dim).drop(dim)
            elif ds[dim].shape == (): ds = ds.drop(dim)
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band')
        except ValueError:
            ds = ds.sum('band')     
    return ds['coolrg']

def pltdata_cooling_1():
    colors = all_palettes['Set1'][4]
    data = [{'label': 'CLIRAD `coolr_bands`',
             'srs': fmt_cool(DS_COOL.sel(i=1)),
             'line_dash': 'dashed', 'line_width': 5,
             'color': colors[0], 'alpha': .6},
            {'label': 'CRD',
             'srs': fmt_cool(DS_COOL_CRD.sum('g')),
             'line_dash': 'solid', 'line_width': 1.5,
             'marker': 'circle', 'marker_size': 5,
             'color': colors[2], 'alpha': 1,},
            {'label': 'WGT igg=10 wgt_flux=2',
             'srs': fmt_cool(DS_COOL_WGT.sel(igg=10).sum('g')),
             'line_dash': 'solid', 'line_width': 3,
             'marker': 'square', 'marker_size': 5,
             'color': colors[3], 'alpha': .6}]
    return data

def nice_xlims(pltdata=None, prange=None):
    def get_slice(srs):
        return srs.sel(pressure=slice(*prange))
    srss = [d['srs'] for d in pltdata]
    vmin = min([get_slice(srs).min() for srs in srss])
    vmax = max([get_slice(srs).max() for srs in srss])
    dv = (vmax - vmin) * .01
    return float(vmin - dv), float(vmax + dv)

def plt_cool(pltdata=None):
    plin = plt_vert_profile_bokeh(pltdata=pltdata,
                                  y_axis_type='linear',
                                  prange=(50, 1050))
    plin.plot_width, plin.plot_height = 400, 500
    plog = plt_vert_profile_bokeh(pltdata=pltdata,
                                  y_axis_type='log',
                                  prange=(.01, 200))
    plog.plot_width, plog.plot_height = 400, 500
    everything = gridplot(plin, plog, ncols=2)
    return everything
    
def script_plt_cooling():
    pltdata = pltdata_cooling_1()
    p = plt_cool(pltdata=pltdata)
    show(p)
    show_markdown('''*FIGURE.* Cooling rate profiles.''')

In [7]:
def fmt_flux(ds_in, ilevels=None):
    ds = ds_in.copy(deep=True)
    
    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    if ilevels: ds = ds.isel(pressure=ilevels)

    dims = ['i', 'igg', 'g']
    for dim in dims:
        if dim in ds:
            if ds[dim].shape == (1,): ds = ds.squeeze(dim).drop(dim)
            elif ds[dim].shape == (): ds = ds.drop(dim)
    
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band').drop('band')
        except ValueError:
            ds = ds.sum('band')
                
    df = ds.to_dataframe()
    df = df.set_index('level', append=True)               
    return df

def tbdata_flux():
    tropopauses = {'mls': 40, 'saw': 45, 'trp': 37}
    atmpro = PARAM['atmpro']
    ilevels = [0, tropopauses[atmpro], -1]
    
    benchmark = {'label': 'CRD',
                 'df': fmt_flux(DS_FLUX_CRD.sum('g'), ilevels=ilevels)}
    
    tbs = [{'label': 'WGT igg=10',
            'df': fmt_flux(DS_FLUX_WGT.sum('g').sel(igg=10), 
                           ilevels=ilevels)},
           {'label': 'CLIRAD',
            'df': fmt_flux(DS_FLUX.sel(i=1), ilevels=ilevels)}]
    
    data = {'benchmark': benchmark, 'others': tbs}
    return data

def tb_flux(tbdata=None):
    benchmark = tbdata['benchmark']
    others = tbdata['others']
    
    def show_tb(d=None):
        display.display(d['df'])
        show_markdown(f'''*TABLE.*  Fluxes. {d['label']}''')
        
    def show_tbdiff(d0=None, d=None):
        df = d['df']
        df.index = d0['df'].index
        display.display(df - d0['df'])
        show_markdown(
            f'''*TABLE.*  Fluxes.  ({d['label']}) - ({d0['label']})''')
        
    show_tb(benchmark)
    for d in others: show_tb(d)
    for d in others: show_tbdiff(d=d, d0=benchmark)
        
def script_tb_flux():
    data = tbdata_flux()
    tb_flux(tbdata=data)

In [8]:
def gasbands():
    bmap = mapband_new2old()
    gases = [gas for gas, _ in PARAM['molecule'].items()] 
    bands = [bmap[b] for b in PARAM['band']]
    return list(itertools.product(gases, bands))

def print_bestfit_params():
    df = pd.DataFrame()
    for gas, band in gasbands():
        param = bestfits.kdist_params(molecule=gas, band=band)
        srs = pd.Series(param)
        df[f'({gas}, {band})'] = srs
    df.fillna('-', inplace=True)
    pd.set_option('display.max_colwidth', -1)
    display.display(df.sort_index())
    pd.reset_option('display.max_colwidth')
        
def print_input_param():
    df_clirad = pd.Series(PARAM, name='clirad run').to_frame()
    df_lblnew = pd.Series(PARAM_LBLNEW, name='lblnew run compared against').to_frame()
    df = pd.merge(df_clirad, df_lblnew, how='outer', 
                  left_index=True, right_index=True)
    df.fillna('-', inplace=True)
    pd.set_option('display.max_colwidth', -1)
    display.display(df)
    pd.reset_option('display.max_colwidth')
        
def print_lblnew_param():
    pd.set_option('display.max_colwidth', -1)
    display.display(pd.Series(PARAM_LBLNEW).to_frame())
    pd.reset_option('display.max_colwidth')

In [9]:
'''
Cooling rate profile by layer
'''
def script_coolr_bylayer():
    '''
    Displays cooling rate of each layer
    '''
    keys = ['CRD', 'CLIRAD `coolr_bands`']
    dss = [DS_COOL_CRD.sum('g'), DS_COOL.sel(i=1)]
    das = [fmt_cool(ds) for ds in dss]
    
    dfs = [da.to_dataframe() for da in das]
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'coolr_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))

In [10]:
'''
Fluxes by layer
'''
def script_flux_bylayer():
    '''
    Display fluxes of each layer
    '''
    keys = ['CRD', 'CLIRAD']
    dss = [DS_FLUX_CRD.sum('g'), DS_FLUX.sel(i=1)]
    dfs = [fmt_flux(ds) for ds in dss]
    
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'fluxes_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))

In [11]:
def script_fluxcoolr_excelfile():
    '''
    Save fluxes and cooling rates by layer by g-interval in excel file
    and make it available for download.
    '''
    fname = 'output_byg.xlsx'
    
    writer = pd.ExcelWriter(fname)
    
    for g in DS_FLUX_WGT.coords['g']:
        df = DS_FLUX_WGT.sel(igg=1).sel(g=g).to_dataframe()
        df.to_excel(writer, f'flux g={int(g)}')
        
    for g in DS_COOL_WGT.coords['g']:
        df = DS_COOL_WGT.sel(igg=1).sel(g=g).to_dataframe()
        df.to_excel(writer, f'cool g={int(g)}')    

In [12]:


    
def script():
    
    title = '### CLIRAD-LW'
    s_input = 'Input Parameters'
    s_bestfits = 'Best-fit Parameters'
    s_plt_cooling = 'Cooling Rate Profiles'
    s_tb_flux = 'Flux Comparison'
    s_coolr_bylayer = 'Cooling Rate by Layer'
    s_flux_bylayer = 'Fluxes by Layer'
    
    # title
    show_markdown('### CLIRAD-LW')
    
    # TOC
    show_markdown('### Table of Contents')
    show_html(climahtml.getHTML_hrefanchor(s_input))
    show_html(climahtml.getHTML_hrefanchor(s_bestfits))
    show_html(climahtml.getHTML_hrefanchor(s_plt_cooling))
    show_html(climahtml.getHTML_hrefanchor(s_tb_flux))
    show_html(climahtml.getHTML_hrefanchor(s_coolr_bylayer))
    show_html(climahtml.getHTML_hrefanchor(s_flux_bylayer))
    
    # Input parameters
    show_html(climahtml.getHTML_idanchor(s_input))
    show_markdown(climahtml.getMarkdown_sectitle(s_input))
    print_input_param()
    
    # Best-fit values
    show_html(climahtml.getHTML_idanchor(s_bestfits))
    show_markdown(climahtml.getMarkdown_sectitle(s_bestfits))
    print_bestfit_params()
    
    # Cooling rate profiles
    show_html(climahtml.getHTML_idanchor(s_plt_cooling))
    show_markdown(climahtml.getMarkdown_sectitle(s_plt_cooling))
    script_plt_cooling()
    
    # Flux comparison
    show_html(climahtml.getHTML_idanchor(s_tb_flux))
    show_markdown(climahtml.getMarkdown_sectitle(s_tb_flux))
    script_tb_flux()
    
    # Cooling rate by layer
    show_html(climahtml.getHTML_idanchor(s_coolr_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_coolr_bylayer))
    script_coolr_bylayer()
    
    # Fluxes by layer
    show_html(climahtml.getHTML_idanchor(s_flux_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_flux_bylayer))
    script_flux_bylayer()

In [13]:
%run param.py

client = pymongo.MongoClient('localhost', 27017)

doc = client.lblnew.bestfit_lw.find_one(make_query(param=PARAM_LBLNEW))
DS_FLUX_CRD = load_output_file(io.StringIO(doc['output_fluxg']))
DS_COOL_CRD = load_output_file(io.StringIO(doc['output_coolrg']))
DS_FLUX_WGT = load_output_file(io.StringIO(doc['output_wfluxg']))
DS_COOL_WGT = load_output_file(io.StringIO(doc['output_wcoolrg']))

doc = client.cliradnew.lw.find_one(make_query(param=PARAM))
DS_FLUX = load_output_file(io.StringIO(doc['output_flux']))
DS_COOL = load_output_file(io.StringIO(doc['output_coolr']))

In [14]:
script()
if 'ng_refs' in PARAM_LBLNEW:
    script_fluxcoolr_excelfile()

### CLIRAD-LW

### Table of Contents

# Input Parameters

,clirad run,lblnew run compared against
atmpro,saw,saw
band,[3],3a
commitnumber,1013f91,a22ab94
conc,-,0.0004
dv,-,0.001
klin,-,6.37556e-24
molecule,"{'co2': 0.0004, 'h2o': 'atmpro', 'n2o': 3.2e-07}",co2
ng_adju,-,"[0, 0, 0]"
ng_refs,-,"[3, 2, 4]"
nv,-,1000


# Best-fit Parameters

...

*TABLE.*  Fluxes.  (CLIRAD) - (CRD)

# Cooling Rate by Layer

CRD           CLIRAD `coolr_bands`          
            layer    coolrg                layer    coolrg
pressure                                                  
0.000312        1  0.188044                    1  0.126876
0.000750        2  0.214588                    2  0.152994
0.001052        3  0.293138                    3  0.213983
0.001476        4  0.379804                    4  0.282248
0.002070        5  0.472872                    5  0.357172
0.002904        6  0.570643                    6  0.438305
0.004074        7  0.670541                    7  0.527983
0.005714        8  0.769042                    8  0.624955
0.008015        9  0.861818                    9  0.725171
0.011243       10  0.943534                   10  0.825118
0.015771       11  1.008420                   11  0.920087
0.022122       12  1.051216                   12  1.003604
0.031031       13  1.068546                   13  1.066317
0.043528       14  1.036762                   14  1.078826
0.061057       15  0.942239                   15  1.017097
0.085645       16  0.823091                   16  0.901697
0.120136       17  0.713015                   17  0.763393
0.168516       18  0.622430                   18  0.629737
0.236378       19  0.558470                   19  0.532887
0.331549       20  0.527826                   20  0.496463
0.465100       21  0.544308                   21  0.530311
0.652400       22  0.491061                   22  0.512727
0.915100       23  0.359247                   23  0.411045
1.283650       24  0.264390                   24  0.322175
1.800600       25  0.206347                   25  0.256372
2.525700       26  0.163279                   26  0.203892
3.542800       27  0.126308                   27  0.163437
4.969550       28  0.100435                   28  0.137863
6.970850       29  0.087924                   29  0.127358
9.778100       30  0.078098                   30  0.115741
13.715850      31  0.066220                   31  0.096570
19.239350      32  0.053203                   32  0.072772
26.987250      33  0.047291                   33  0.057862
37.855300      34  0.048736                   34  0.054560
53.100050      35  0.049165                   35  0.056814
73.887500      36  0.048824                   36  0.062694
97.662500      37  0.048020                   37  0.065291
121.437500     38  0.044461                   38  0.061138
145.212500     39  0.039541                   39  0.054251
168.987500     40  0.035310                   40  0.047832
192.762500     41  0.031101                   41  0.041736
216.537500     42  0.026651                   42  0.035582
240.312500     43  0.021669                   43  0.028874
264.087500     44  0.015451                   44  0.020860
287.862500     45  0.007170                   45  0.010930
311.637500     46  0.004500                   46  0.008184
335.412500     47  0.012056                   47  0.018128
359.187500     48  0.021783                   48  0.031610
382.962500     49  0.029550                   49  0.046052
406.737500     50  0.036388                   50  0.062232
430.512500     51  0.042684                   51  0.078758
454.287500     52  0.048586                   52  0.095321
478.062500     53  0.054169                   53  0.111402
501.837500     54  0.059460                   54  0.125859
525.612500     55  0.064458                   55  0.138267
549.387500     56  0.069116                   56  0.149525
573.162500     57  0.073369                   57  0.160425
596.937500     58  0.077126                   58  0.170814
620.712500     59  0.080298                   59  0.179671
644.487500     60  0.082830                   60  0.186479
668.262500     61  0.084719                   61  0.190716
692.037500     62  0.086063                   62  0.192691
715.812500     63  0.087001                   63  0.192981
739.587500     64  0.087736                   64  0.192242
763.362500     65  0.088499                   65  0.191087
78

# Fluxes by Layer

CRD                          CLIRAD                \
                        flug      fldg      fnetg       flug          fldg   
pressure    level                                                            
0.000000    1     -20.835452  0.000000 -20.835452 -19.778943  4.523207e-08   
0.000624    2     -20.835486  0.000048 -20.835438 -19.778971  3.762316e-05   
0.000876    3     -20.835500  0.000069 -20.835431 -19.778982  5.347297e-05   
0.001229    4     -20.835519  0.000100 -20.835419 -19.778998  7.790425e-05   
0.001723    5     -20.835545  0.000148 -20.835397 -19.779019  1.155055e-04   
0.002417    6     -20.835580  0.000222 -20.835358 -19.779047  1.731826e-04   
0.003391    7     -20.835627  0.000334 -20.835292 -19.779085  2.613448e-04   
0.004757    8     -20.835688  0.000504 -20.835184 -19.779134  3.962893e-04   
0.006672    9     -20.835768  0.000758 -20.835009 -19.779199  6.023637e-04   
0.009359    10    -20.835869  0.001134 -20.834735 -19.779280  9.147715e-04   
0.013128    11    -20.835996  0.001682 -20.834314 -19.779381  1.383884e-03   
0.018415    12    -20.836149  0.002467 -20.833682 -19.779501  2.079776e-03   
0.025830    13    -20.836325  0.003566 -20.832759 -19.779635  3.095640e-03   
0.036232    14    -20.836512  0.005070 -20.831442 -19.779771  4.545815e-03   
0.050823    15    -20.836696  0.007045 -20.829650 -19.779893  6.533195e-03   
0.071291    16    -20.836873  0.009507 -20.827366 -19.779995  9.100957e-03   
0.100000    17    -20.837038  0.012471 -20.824567 -19.780059  1.223217e-02   
0.140271    18    -20.837152  0.015986 -20.821165 -19.780042  1.585711e-02   
0.196760    19    -20.837144  0.020144 -20.817000 -19.779866  1.989518e-02   
0.275997    20    -20.836876  0.025118 -20.811758 -19.779391  2.442219e-02   
0.387100    21    -20.836036  0.031224 -20.804812 -19.778317  2.988259e-02   
0.543100    22    -20.833675  0.038922 -20.794753 -19.775760  3.712660e-02   
0.761700    23    -20.829442  0.047405 -20.782037 -19.771336  4.598106e-02   
1.068500    24    -20.824449  0.055467 -20.768981 -19.766023  5.560804e-02   
1.498800    25    -20.819235  0.063730 -20.755505 -19.760329  6.633782e-02   
2.102400    26    -20.813833  0.073082 -20.740751 -19.754303  7.864446e-02   
2.949000    27    -20.808389  0.084013 -20.724376 -19.748141  9.293164e-02   
4.136600    28    -20.803497  0.096890 -20.706607 -19.742425  1.102112e-01   
5.802500    29    -20.799655  0.112868 -20.686788 -19.737653  1.326472e-01   
8.139200    30    -20.796551  0.134100 -20.662451 -19.733538  1.637877e-01   
11.417000   31    -20.794203  0.162076 -20.632127 -19.730282  2.054769e-01   
16.014700   32    -20.793836  0.197774 -20.596062 -19.729585  2.573796e-01   
22.464000   33    -20.798642  0.243226 -20.555416 -19.735470  3.188662e-01   
31.510500   34    -20.810782  0.306044 -20.504738 -19.750645  3.960540e-01   
44.200100   35    -20.829451  0.397971 -20.431479 -19.773935  5.013653e-01   
62.000000   36    -20.855990  0.528176 -20.327814 -19.807247  6.544831e-01   
85.775000   37    -20.891006  0.700694 -20.190311 -19.851661  8.754811e-01   
109.550000  38    -20.925449  0.870377 -20.055072 -19.895703  1.103422e+00   
133.325000  39    -20.961979  1.032122 -19.929857 -19.942353  1.322272e+00   
157.100000  40    -21.003750  1.185253 -19.818497 -19.995407  1.528131e+00   
180.875000  41    -21.051398  1.332346 -19.719052 -20.055656  1.723103e+00   
204.650000  42    -21.105671  1.474211 -19.631460 -20.124261  1.909262e+00   
228.425000  43    -21.167672  1.611270 -19.556401 -20.203070  2.088292e+00   
252.200000  44    -21.239189  1.743814 -19.495376 -20.294592  2.261141e+00   
275.975000  45    -21.324009  1.872147 -19.451862 -20.403511  2.428815e+00   
299.750000  46    -21.430450  1.998780 -19.431670 -20.538571  2.594659e+00   
323.525000  47    -21.558621  2.139623 -19.418998 -20.697899  2.777038e+00   
347.300000  48    -21.696831  2.311785 -19.385046 -20.866477  2.996675e+00   
371.075000  49    -21.837747  2.514048 -19.3236

In [15]:
display.HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')